Learning F# 

In [1]:
let generateSquare (min:int) (max:int): int seq = 
    Seq.append {(max-1)..(-1)..min} {(min+1)..max}
    |> (fun x -> seq { for i in 1..4 do yield! x } )

let rec squareBuilder ((goal, acc, min, max):int*int*int*int) : int seq =
    match (goal <= acc) with 
    | true -> Seq.empty
    | _ -> 
        let newSquare = generateSquare min max
        squareBuilder (goal, (acc + Seq.length newSquare), (min+1), (max+2))
        |> Seq.append newSquare
    
let adventOfCode (goal:int) : int = 
   
    squareBuilder (goal, 0, 1, 2)
    |> Seq.toList
    |> (fun distanceList -> distanceList.[goal-2])
    


In [2]:
adventOfCode 277678

475

In [3]:
squareBuilder (100, 0, 1, 2) |> Seq.toList

[1; 2; 1; 2; 1; 2; 1; 2; 3; 2; 3; 4; 3; 2; 3; 4; 3; 2; 3; 4; 3; 2; 3; 4; 5; 4; 3;
 4; 5; 6; 5; 4; 3; 4; 5; 6; 5; 4; 3; 4; 5; 6; 5; 4; 3; 4; 5; 6; 7; 6; 5; 4; 5; 6;
 7; 8; 7; 6; 5; 4; 5; 6; 7; 8; 7; 6; 5; 4; 5; 6; 7; 8; 7; 6; 5; 4; 5; 6; 7; 8; 9;
 8; 7; 6; 5; 6; 7; 8; 9; 10; 9; 8; 7; 6; 5; 6; 7; 8; 9; 10; ...]

In [4]:
type Movement = Up | Left | Down | Right

In [5]:
let advanceMovement (movement:Movement) : Movement =
    match movement with
    | Up -> Left
    | Left -> Down
    | Down -> Right
    | Right -> Up
    

In [6]:
type Point = int*int

In [7]:
type Item = Point*int

In [8]:
let findVal (items:Item seq) (point:Point) : int =
    items
    |> Seq.find (fun (item, _) -> if item = point then true else false) 
    |> (fun (_, num) -> num)

In [9]:
let getVal ((x, y):Point) (items:Item seq) : Item = 
    seq [(x+1, y); (x+1, y+1); (x, y+1); (x-1, y+1); (x-1, y); (x-1, y-1); (x, y-1); (x+1, y-1);]
    |> Seq.map (findVal items)
    |> Seq.sum
    |> (fun num -> (x,y), num)
    

In [10]:
let updatePoint (movement:Movement) (point:Point) : Point =
    match movement with
    | Up -> point |> (fun (x, y) -> x, y+1)
    | Left -> point |> (fun (x, y) -> x-1, y)
    | Down -> point |> (fun (x, y) -> x, y-1)
    | Right -> point |> (fun (x, y) -> x+1, y)

In [11]:
let subFolder ((moveState, itemState):Movement*Item seq) (num:int) : Movement*Item seq =
    
    let newPoint = 
        Seq.last itemState 
        |> (fun (point, _) -> point)
        |> updatePoint moveState
    
    let newItem = 
        getVal newPoint itemState
    
    newItem
    |> Seq.singleton
    |> Seq.append itemState
    |> (fun x -> (moveState, x))


In [12]:
let folder (num:int) ((moveState, pointState):Movement*Item seq)  : Movement*Item seq =
    
    let (fstMove, fstPoints) = Seq.fold subFolder (advanceMovement moveState, pointState) {1..num}
    
    let (newMove, sndPoints) = Seq.fold subFolder (advanceMovement fstMove, fstPoints) {1..num}
    
    (newMove, sndPoints)
    
    

In [21]:
let adventOfCode3 (limit:int) : Item = 
    let infiniSeq = Seq.initInfinite (fun index -> index + 1)
    
    let helper ((_, num):Item) : bool = 
        if num > limit then 
            true 
        else 
            false
    
    (Down, Seq.singleton ((0,0), 0))
    |> Seq.foldBack folder infiniSeq
    |> (fun (_, items) -> Seq.find helper items)


In [ ]:
adventOfCode3 5